In [2]:
import sys
import os

# Add the path to the aqi package
aqi_path = os.path.abspath('../../../airflow')
sys.path.append(aqi_path)

from plugins.hooks.station import fetch_data
from dags.etl.station import validate_data, parse_data, clean_data, push_to_db

In [3]:
aqi_data = fetch_data()
print(aqi_data['status'])
print(len(aqi_data['data']))

<Response [200]>
{'status': 'ok', 'data': [{'lat': 14.897273, 'lon': 105.475778, 'uid': -421321, 'aqi': '38', 'station': {'name': 'เขตรักษาพันธุ์สัตว์ป่าบุณฑริก-ยอดมน', 'time': '2025-02-25T11:28:21+09:00'}}, {'lat': 15.206423, 'lon': 100.151848, 'uid': -419392, 'aqi': '38', 'station': {'name': 'สวนนกชัยนาท', 'time': '2025-02-25T11:28:23+09:00'}}, {'lat': 16.683188, 'lon': 100.100324, 'uid': -419251, 'aqi': '31', 'station': {'name': 'เทศบาลตำบลปลักแรด', 'time': '2025-02-25T11:09:49+09:00'}}, {'lat': 16.44538333, 'lon': 102.83525, 'uid': 3688, 'aqi': '59', 'station': {'name': 'Khonkaen, Thailand (ส่วนอุทกวิทยา สำนักงานทรัพยากรน้ำภาค 4 ขอนแก่น)', 'time': '2025-02-25T11:00:00+09:00'}}, {'lat': 12.73518333, 'lon': 101.13565, 'uid': 1821, 'aqi': '69', 'station': {'name': 'Field Crop Research Center, Rayong, Thailand (ศูนย์วิจัยพืชไร่ระยอง)', 'time': '2025-02-25T11:00:00+09:00'}}, {'lat': 8.426, 'lon': 99.512, 'uid': -86866, 'aqi': '33', 'station': {'name': 'Cha Wang', 'time': '2025-02-25T11:

In [4]:
validate_data(aqi_data)

In [5]:
parse_aqi_data = parse_data(aqi_data)
print(parse_aqi_data[:3])

      uid aqi   latitude   longitude                         station_name  \
0 -421321  38  14.897273  105.475778  เขตรักษาพันธุ์สัตว์ป่าบุณฑริก-ยอดมน   
1 -419392  38  15.206423  100.151848                          สวนนกชัยนาท   
2 -419251  31  16.683188  100.100324                    เทศบาลตำบลปลักแรด   

                  timestamp  
0 2025-02-25 11:28:21+09:00  
1 2025-02-25 11:28:23+09:00  
2 2025-02-25 11:09:49+09:00  


In [12]:
clean_aqi_data = clean_data(parse_aqi_data)
print(clean_aqi_data[:3])

      uid   aqi   latitude   longitude  \
0    9014  58.0  13.917563  101.590529   
1 -421522  76.0  13.852946  100.561281   
2   10481  84.0  14.022376   99.536195   

                                        station_name                 timestamp  
0  Tha Tum, Si Maha Phot, Prachin Buri, Thailand ... 2025-02-24 23:00:00+09:00  
1                     บริษัท ดีดีมาร์ท โฮลดิ้ง จำกัด 2025-02-24 23:18:26+09:00  
2  Pak Prak, Kanchanaburi, Kanchanaburi, Thailand... 2025-02-24 22:00:00+09:00  


In [10]:
import psycopg2

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Push the cleaned AQI data to the database
push_data = push_to_db(conn, clean_aqi_data)

# Close the connection
conn.close()